# Change Detection (Post-classification Comparison)


- **Special requirements:** A Google account, access to Google Earth Engine.

- **Prerequisites:** You should have completed the "Week 4 - Prac 1" notebook.

## Description
Many earth observation datasets are available at regular intervals over long periods of time. This enables us to detect changes on the Earth’s surface. In this session, you will learn how to apply change detection technique in remote sensing.

## Aims of the practical session
* Create ROI and load it
* Collect images and training samples
* Apply classification algorithms to classify images before and after an event
* Comparing two classified images to see the changes during the time

## Getting started

### Load packages

Import GEE packages that are needed for the analysis.

In [1]:
import ee
import geemap
# ee.Authenticate()

### Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [3]:
import geemap
Map = geemap.Map()
# Map.add_basemap('HYBRID')

### Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map.
Import the downloaded shapefile for Canberra central from your computer path as ROI.

In [ ]:
# # # load shapefile (ROI)
path = 'https://raw.githubusercontent.com/nicolasyounes/engn3903/week5update/figures/act_localities.geojson'
geometry = geemap.shp_to_ee(shp_path)
Map.addLayer(geometry, {}, 'Canberra central SHP')
Map.centerObject(geometry);

### Training data
Training data (or a training dataset) is the initial data used to train machine learning models. Import your pre-selected training dataset from your system.

In [ ]:
#### load training data
training_path = "https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/training_data.geojson"
training_data = geemap.geojson_to_ee(training_path)

#print how many classes there are in the TD
df = geemap.ee_to_geopandas(training_data)
n_classes = len(df['landcover'].unique())
print(f'There are {n_classes} landcover classes in the training dataset')

### Before Image collection 

In the next few code cells we will extract training data from Sentinel-2 images over the pixels specified by the training sample locations loaded from the URL page.

Sentinel-2 is a wide-swath, high-resolution, multi-spectral imaging mission supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas.

We will:
* Define a function for cloud masing and resabling sentinel-2 images
* Load Sentinel-2 images for the analysis
* Filter a collection by date range
* Calculate a temporal median to collapse the time dimension
* Clip based on the geometry

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask).divide(10000) #re-scale 

In [ ]:
## collect before event images 
S2_before = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate('2019-10-01', '2019-10-31') ### Note: you can try different dates
    .map(maskS2clouds) #map the cloudmasking/rescaling function
    .median()
    .clip(geometry)
)

In [ ]:
# # select bands wanted to use in the classification
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

### Visualize the composite
* Clip based on the geometry
* Display it on Geemap

In [ ]:
vis_params = {'min': 0, 'max': 0.4, 'bands': ['B4', 'B3', 'B2']}

#Map.centerObject(point, 8)
Map.addLayer(S2_before, vis_params, "Sentinel2_before")
Map

### Overlay the point on the image to get training data
Now that we have created the points and labels, we need to sample the imagery using image.sampleRegions(). This command will extract the reflectance in the designated bands for each of the points you have created. 

In [ ]:
# Overlay the point on the image to get training data.
training = S2_before.sampleRegions(**{
  'collection': training_data,
  'properties': ['landcover'],
  'scale': 20
})

### Classifcation method
The <a href="https://developers.google.com/earth-engine/guides/classification">Classifier</a> package in handles supervised classification by ML algorithms running in Earth Engine. Thus, in this part we will:
* Instantiate a supervised classifier
* Set its parameters if necessary
* Train the classifier using the training data
* Classify an image or feature collection
* Display the classified map

In [ ]:
# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': bands
})

In [ ]:
# # Classify the image.
beforeClassified = S2_before.classify(classifier)
Map.addLayer(beforeClassified,{'min': 0, 'max': 3, 'palette': ['blue', 'green', 'yellow', 'red']}, 'before_classified')
Map

### After Image collection 
We will:
* Load after Landsat-8 images for the anlysis
* Filter a collection by date range
* Make a cloud-free composite 

In [ ]:
# # After Images.
S2_after = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate('2021-10-01', '2021-10-31') ### Note: you can try different dates
    .map(maskS2clouds) #map the cloudmasking/rescaling function
    .median()
    .clip(geometry)
)

In [ ]:
# # select bands wanted to use in the classification
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

### Visualize the composite
* Clip based on the geometry
* Display it on Geemap

In [ ]:
# Visualize the Composite
vis_params = {'min': 0, 'max': 0.4, 'bands': ['B4', 'B3', 'B2']}

# Map.centerObject(point, 8)
Map.addLayer(S2_after, vis_params, "Sentinel2_after")
Map.addLayerControl()
Map

### Classify the image
* Apply the same supervised classifier on after image
* Classify an image or feature collection
* Display the classified map

In [ ]:
# Classify the image.
afterClassified= S2_after.classify(classifier)
Map.addLayer(afterClassified,{'min': 0, 'max': 3, 'palette': ['blue', 'green', 'yellow', 'red']}, 'after_classified')
Map

<div class="alert alert-block alert-danger"> 

### Exercise 1: Accuracy Assessment

<div class="alert alert-block alert-danger">

- Try to calculate the accuracy of the method for both pre and post-classifcation results to see if the performance of the model is satisfactory and compare the results. 
    
    
</div> 

In [ ]:
# Reclassify from 0-3 to 1-4
beforeClasses = beforeClassified.remap([0, 1, 2, 3], [1, 2, 3, 4])
afterClasses = afterClassified.remap([0, 1, 2, 3], [1, 2, 3, 4])

### Post-classification comparison
We dealing with multi-class images, a useful metric for change detection is to know how many pixels from class X changed to class Y. 

In [ ]:
# Show all changed areas
changed = afterClasses.subtract(beforeClasses).neq(0)
Map.addLayer(changed, {'min':0, 'max':1, 'palette': ['white', 'red']}, 'Change')
Map

<div class="alert alert-block alert-danger"> 
    
### Exercise 2 - Calculate area by each transition class 
Try to calculate area covered by each class in a classified images (before and after) and display it in a chart.

#### Then try to check the following exercises after applying change detection:
* Calculate the area of each class in each classifcation.
* Discuss if the changes involve a decrease in a class, lead to an increase, or keep unchanged. 
<div>

## References
This is where the references go. For exmaple:

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
* "Earth Observation: Data, Processing and Applications" book. Available through Wattle, or http://www.crcsi.com.au/earth-observation-series.

## Additional information

**License:** The code in this notebook was initially created by the team at [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks), and has been modified by Abolfazl Abdollahi. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle site.

**Last modified:** August 2022